**SPAM HAM** **Classifier**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re 
import string
import nltk 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#messages = pd.read_csv('spam.csv')
df = pd.read_csv('spam.csv', encoding='latin-1')
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)


In [3]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
def cleaning_reviews(df):
    messages = list()
    lines = df['v2'].values.tolist()
    for i in lines:
        text = i.lower()
        pattern = re.compile('http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub("", text)
        tokens = nltk.word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_word = set(stopwords.words('english'))
        stop_word.discard("not")
        words = [nltk.WordNetLemmatizer().lemmatize(w) for  w in words if not w in stop_word]
        words = ' '.join(words)
        messages.append(words)

    return(messages)

corpus = cleaning_reviews(df)

In [5]:
corpus.__len__()

5572

In [6]:
corpus[0:5]

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah nt think go usf life around though']

In [7]:
df.v1.value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [8]:
CV = CountVectorizer(max_features= 500)
X = CV.fit_transform(corpus).toarray()
Y=pd.get_dummies(df['v1'])
Y = Y.iloc[:, 1].values

In [9]:
print(X[0:3])
print(Y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0 0 1 ... 0 0 0]


In [10]:
feature_names = CV.get_feature_names_out()
print(feature_names.__len__())
print(feature_names)

500
['able' 'abt' 'account' 'actually' 'address' 'aft' 'afternoon' 'age' 'ah'
 'aight' 'already' 'alright' 'also' 'always' 'amp' 'angry' 'another'
 'answer' 'anything' 'anyway' 'apply' 'ard' 'around' 'ask' 'asked'
 'attempt' 'auction' 'await' 'award' 'awarded' 'away' 'awesome' 'babe'
 'baby' 'back' 'bad' 'beautiful' 'bed' 'believe' 'best' 'better' 'big'
 'birthday' 'bit' 'bonus' 'book' 'bored' 'box' 'boy' 'break' 'bring'
 'brother' 'bt' 'bus' 'busy' 'buy' 'ca' 'call' 'called' 'calling' 'came'
 'camera' 'cant' 'car' 'card' 'care' 'cash' 'cause' 'chance' 'change'
 'charge' 'chat' 'check' 'chikku' 'claim' 'class' 'close' 'club' 'co'
 'code' 'collect' 'collection' 'colour' 'come' 'coming' 'company'
 'congrats' 'contact' 'content' 'cool' 'cost' 'could' 'coz' 'credit'
 'customer' 'da' 'dad' 'dat' 'date' 'day' 'de' 'dear' 'decimal' 'delivery'
 'den' 'detail' 'didnt' 'dinner' 'dis' 'done' 'dont' 'double' 'draw'
 'dream' 'drink' 'drive' 'driving' 'drop' 'dude' 'dun' 'dunno' 'early'
 'easy' 'eat

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [12]:
from sklearn.naive_bayes import MultinomialNB
spam_detect = MultinomialNB().fit(X_train, Y_train)
Y_pred = spam_detect.predict(X_test)
print(accuracy_score(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred))
print(precision_score(Y_test, Y_pred))

0.965311004784689
0.8632075471698113
0.8926829268292683
